In [1]:
year = 2019
month = 1
start_date = None
end_date = None
program = 'TOPC1451546588784893'
# TOPC1451557646802924

In [2]:
import aiohttp

In [3]:
import asyncio
import json
import pandas as pd
import re
import papermill as pm
import datetime

In [4]:
target_date = datetime.date(year=year, month=month, day=1)

In [5]:
target_date = target_date.strftime("%Y%m")

In [6]:
target_date

'201901'

In [7]:
async def fetch(session, url):
    async with session.get(url) as response:
        return await response.text()

In [8]:
async with aiohttp.ClientSession() as session:
    json_str = await fetch(session, f"http://api.cntv.cn/lanmu/videolistByColumnId?id={program}&n=90&of=fdate&p=1&type=0&serviceId=tvcctv&d={target_date}")

In [9]:
json_data = json.loads(json_str)['response']['docs']

In [10]:
json_str

'{"response":{"start":0,"docs":[{"videoBrief":"本期节目主要内容：  圈友求助，猪油能不能吃？达人支招，教你轻松灌出美味香肠；圈友求助，用什么面粉做饺子皮更好吃？圈友求助，做馒头怎样和面更均匀？温暖回家路，西安开行首趟“暖心高铁专列”；关注身边的安全，安全用电，平安过年。 （《生活圈》 20190131）","videoThirdclass":"","videoSecondclass":"饮食,健康,居家,百科","videoUrl":"http://tv.cctv.com/2019/01/31/VIDE80RtHHkDFFLZI2MznLqC190131.shtml","videoEditMode":"0","videoTitle":"《生活圈》 20190131","videoFirstclass":"生活","videoPlaystyle":"","videoFocusDate":"1548902205000","videoKeyFrameUrl":"http://p1.img.cctvpic.com/fmspic/2019/01/31/ddb10cebb08c400f9d9dfd31176c4bef-1270.jpg","videoMediaType":"","videoPublishFlg":"1","videoTag":"生活圈，闻阳，猪油，饺子皮，“暖心高铁专列”，安全用电","videoSharedCode":"ddb10cebb08c400f9d9dfd31176c4bef","videoChannel":"","videoId":"VIDE80RtHHkDFFLZI2MznLqC190131","videoLength":"00:41:33","videoLastmodifyDate":"1548925485000","videoDeleteFlg":"0","videoProductiontime":"1548901265000","videoKeyFrameUrl2":"","videoKeyFrameUrl3":""},{"videoBrief":"本期节目主要内容：\\n1、大厨支招：如何做出美味的猪肉白菜馅饺子、素馅饺子和羊肉酸菜馅饺子；\\n2、怎么做鱼才能鲜？\\n3、大厨解答：猪肉、牛

In [11]:
df = pd.DataFrame.from_records(json_data)

In [12]:
df.columns

Index(['videoBrief', 'videoChannel', 'videoDeleteFlg', 'videoEditMode',
       'videoFirstclass', 'videoFocusDate', 'videoId', 'videoKeyFrameUrl',
       'videoKeyFrameUrl2', 'videoKeyFrameUrl3', 'videoLastmodifyDate',
       'videoLength', 'videoMediaType', 'videoPlaystyle',
       'videoProductiontime', 'videoPublishFlg', 'videoSecondclass',
       'videoSharedCode', 'videoTag', 'videoThirdclass', 'videoTitle',
       'videoUrl'],
      dtype='object')

### 从标题中抽取，移除所有非数字 ###

In [13]:
df['videoRecordDay'] = df['videoTitle'].map(lambda x: re.sub("[^0-9]", "", x)).apply(pd.to_datetime)

In [14]:
df.set_index('videoRecordDay', inplace=True)

In [15]:
df.sort_index(inplace=True)

In [16]:
df.loc[start_date:end_date, 'videoUrl']

videoRecordDay
2019-01-02    http://tv.cctv.com/2019/01/02/VIDEyG4W0kxrjaIi...
2019-01-03    http://tv.cctv.com/2019/01/03/VIDEKLFHbC8K2AO7...
2019-01-04    http://tv.cctv.com/2019/01/04/VIDEhqTlEg7Vs3rN...
2019-01-07    http://tv.cctv.com/2019/01/07/VIDEZlbvgMhAbBQf...
2019-01-08    http://tv.cctv.com/2019/01/08/VIDED9IYp2KYNiEo...
2019-01-09    http://tv.cctv.com/2019/01/09/VIDEWMdm3FR4hR7R...
2019-01-10    http://tv.cctv.com/2019/01/10/VIDEHgT9HN8RoZ40...
2019-01-11    http://tv.cctv.com/2019/01/11/VIDEEr4DgPMbKw8g...
2019-01-14    http://tv.cctv.com/2019/01/14/VIDEBNA3fbxJdfFS...
2019-01-15    http://tv.cctv.com/2019/01/15/VIDE7RLzIYnkGQR6...
2019-01-16    http://tv.cctv.com/2019/01/16/VIDETkyAL5o8eDCt...
2019-01-17    http://tv.cctv.com/2019/01/17/VIDEjeV5I2bBRvDT...
2019-01-18    http://tv.cctv.com/2019/01/18/VIDE3bHU3TvFnwNE...
2019-01-21    http://tv.cctv.com/2019/01/21/VIDEm196PO0iAXQ4...
2019-01-22    http://tv.cctv.com/2019/01/22/VIDE583tr4DTBOdM...
2019-01-23    http://tv.c

In [17]:
pm.record("dn_list", df.loc[start_date:end_date, 'videoUrl'].tolist())